### Installation

In [ ]:
!pip install pymupdf
!pip install Unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 23.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 4.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!sudo apt-get install unzip
!unzip -q "/content/gdrive/My Drive/CUAD_v1.zip"

Mounted at /content/gdrive
Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-25ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from operator import itemgetter
import fitz
import json

In [ ]:
# load the file
my_path = '/content/CUAD_v1/full_contract_pdf/Part_I/Affiliate_Agreements/UsioInc_20040428_SB-2_EX-10.11_1723988_EX-10.11_Affiliate Agreement 2.pdf'
doc = fitz.open(my_path)

In [ ]:
for page in doc:
    text = page.get_text()
    print(text)

                                                                   EXHIBIT 10.11
                         NETWORK 1 FINANCIAL CORPORATION
                           AFFILIATE OFFICE AGREEMENT
THIS  AGREEMENT  is  entered  into  by  and  between  NETWORK  1 FINANCIAL, INC.
("NETWORK  1"),  a  Virginia Corporation with its principal place of business at
1501  Farm  Credit  Drive,  Suite 1500, McLean, Virginia 22102-5004, and Payment
Data  Systems,  Inc.,  the  Affiliate Office ("AFFILIATE"), a Nevada Corporation
with  its  principal place of business at 12500 San Pedro Suite 120 San Antonio,
TX  78216.  NETWORK  1  and  Affiliate  hereby  agree  as  follows:
                                    RECITALS
WHEREAS, Network 1 Financial, Inc. ("Network 1") provides through various Member
Bank(s)  ("Member"),  VISA  and  MasterCard  processing  and  related  payment
processing  services  ("Services")  to merchant(s) ("Merchant(s)") in accordance
with  the  terms  of  certain  Agreement (s) betw

In [ ]:
# separate to text blocks and get text data in Unicode
from unidecode import unidecode 

output = []

for page in doc:
    output += page.get_text("blocks")
previous_block_id = 0 # Set a variable to mark the block id
for block in output:
     if block[6] == 0: # only take the text
          if previous_block_id != block[5]: # Compare the block number 
              print("\n")
          plain_text = unidecode(block[4])
          print(plain_text)

                                                                   EXHIBIT 10.11



                         NETWORK 1 FINANCIAL CORPORATION



                           AFFILIATE OFFICE AGREEMENT



THIS  AGREEMENT  is  entered  into  by  and  between  NETWORK  1 FINANCIAL, INC.
("NETWORK  1"),  a  Virginia Corporation with its principal place of business at
1501  Farm  Credit  Drive,  Suite 1500, McLean, Virginia 22102-5004, and Payment
Data  Systems,  Inc.,  the  Affiliate Office ("AFFILIATE"), a Nevada Corporation
with  its  principal place of business at 12500 San Pedro Suite 120 San Antonio,
TX  78216.  NETWORK  1  and  Affiliate  hereby  agree  as  follows:



                                    RECITALS



WHEREAS, Network 1 Financial, Inc. ("Network 1") provides through various Member
Bank(s)  ("Member"),  VISA  and  MasterCard  processing  and  related  payment
processing  services  ("Services")  to merchant(s) ("Merchant(s)") in accordance
with  the  terms  of  certain  Agr

In [ ]:
# get the spans from the PDF file
block_dict = {}

page_num = 1
for page in doc: # Iterate all pages in the document
      file_dict = page.get_text('dict') # Get the page dictionary 
      block = file_dict['blocks'] # Get the block information
      block_dict[page_num] = block # Store in block dictionary
      page_num += 1 # Increase the page value by 1

In [ ]:
# create a dictionary containing detailed information of all spans in the document and store in a DataFrame
import re 

spans = pd.DataFrame(columns=['xmin', 'ymin', 'xmax', 'ymax', 'text', 'tag'])
rows = []

for page_num, blocks in block_dict.items():
    for block in blocks:
        if block['type'] == 0:
            for line in block['lines']:
                for span in line['spans']:
                    xmin, ymin, xmax, ymax = list(span['bbox'])
                    font_size = span['size']
                    text = unidecode(span['text'])
                    span_font = span['font']
                    
                    is_upper = False
                    is_bold = False                     

                    if "bold" in span_font.lower():
                        is_bold = True 
                    if re.sub("[\(\[].*?[\)\]]", "", text).isupper():
                        is_upper = True                 
                    if text.replace(" ","") !=  "":
                        rows.append((xmin, ymin, xmax, ymax, text, is_upper, is_bold, span_font, font_size))
                        
span_df = pd.DataFrame(rows, columns=['xmin','ymin','xmax','ymax', 'text', 'is_upper','is_bold','span_font', 'font_size'])
span_df

,xmin,ymin,xmax,ymax,text,is_upper,is_bold,span_font,font_size
0,49.633026,50.317444,420.275452,58.846008,...,True,False,CourierNewPSMT,7.528665
1,49.633026,68.849426,309.083191,77.377991,NETWORK 1 FINANCIAL C...,True,False,CourierNewPSMT,7.528665
2,49.633026,87.381653,295.184082,95.910217,AFFILIATE OFFICE AG...,True,False,CourierNewPSMT,7.528665
3,49.633026,105.913635,420.275452,114.442200,THIS AGREEMENT is entered into by and b...,False,False,CourierNewPSMT,7.528665
4,49.633026,115.179749,420.275452,123.708313,"(""NETWORK 1""), a Virginia Corporation with ...",False,False,CourierNewPSMT,7.528665
...,...,...,...,...,...,...,...,...,...
394,49.633026,64.213730,183.991425,72.742294,Network 1 Affiliate,False,False,CourierNewPSMT,7.528665
395,49.633026,73.479782,285.917969,82.008347,By: ______________________________________ ...,False,False,CourierNewPSMT,7.528665
396,49.633026,82.745834,234.954712,91.274399,________________________________________,False,False,CourierNewPSMT,7.528665
397,49.633026,92.011879,313.716125,100.540443,Authorized Representative Authorized Rep...,False,False,CourierNewPSMT,7.528665


In [ ]:
# create more features
"""
The ‘h’ tag denotes the text which is bigger and more important than normal paragraphs. (e.g. UPPER CASE and bold style)
The ‘p’ tag stands for paragraph, or the normal content in the document. 
The ‘s’ tag will be used for less important text, which is smaller than ‘p’ text.
"""

span_scores = []
span_num_occur = {}
special = '[(_:/,#%\=@)]'

for index, span_row in span_df.iterrows():    
    score = round(span_row.font_size)
    text = span_row.text    
    if not re.search(special, text):       
        if span_row.is_bold:
            score +=1 
        if span_row.is_upper:
            score +=1
    span_scores.append(score)

values, counts = np.unique(span_scores, return_counts=True)
style_dict = {}

for value, count in zip(values, counts):
    style_dict[value] = count

sorted(style_dict.items(), key=lambda x: x[1])

[(10, 8), (9, 40), (8, 351)]

In [ ]:
p_size = max(style_dict, key=style_dict.get)
idx = 0
tag = {}

for size in sorted(values, reverse = True):
    idx += 1
    if size == p_size:
        idx = 0
        tag[size] = 'p'
    if size > p_size:
        tag[size] = 'h{0}'.format(idx)
    if size < p_size:
        tag[size] = 's{0}'.format(idx)

In [ ]:
span_tags = [tag[score] for score in span_scores]
span_df['tag'] = span_tags

span_df

,xmin,ymin,xmax,ymax,text,is_upper,is_bold,span_font,font_size,tag
0,49.633026,50.317444,420.275452,58.846008,...,True,False,CourierNewPSMT,7.528665,h2
1,49.633026,68.849426,309.083191,77.377991,NETWORK 1 FINANCIAL C...,True,False,CourierNewPSMT,7.528665,h2
2,49.633026,87.381653,295.184082,95.910217,AFFILIATE OFFICE AG...,True,False,CourierNewPSMT,7.528665,h2
3,49.633026,105.913635,420.275452,114.442200,THIS AGREEMENT is entered into by and b...,False,False,CourierNewPSMT,7.528665,p
4,49.633026,115.179749,420.275452,123.708313,"(""NETWORK 1""), a Virginia Corporation with ...",False,False,CourierNewPSMT,7.528665,p
...,...,...,...,...,...,...,...,...,...,...
394,49.633026,64.213730,183.991425,72.742294,Network 1 Affiliate,False,False,CourierNewPSMT,7.528665,p
395,49.633026,73.479782,285.917969,82.008347,By: ______________________________________ ...,False,False,CourierNewPSMT,7.528665,p
396,49.633026,82.745834,234.954712,91.274399,________________________________________,False,False,CourierNewPSMT,7.528665,p
397,49.633026,92.011879,313.716125,100.540443,Authorized Representative Authorized Rep...,False,False,CourierNewPSMT,7.528665,p


In [ ]:
# define headings and paragraphs in the document
headings_list = []
text_list = []
tmp = []
heading = ''                                                                                                                

for index, span_row in span_df.iterrows():
    text = span_row.text
    tag = span_row.tag
    if 'h' in tag:
      if len(text) < 70:
        headings_list.append(text)
        text_list.append('\n'.join(tmp)) 
        tmp = []
        heading = text
      else:
        tmp.append(text) 
    else:
        tmp.append(text)

text_list.append('\n'.join(tmp))
text_list = text_list[1:]
text_df = pd.DataFrame(zip(headings_list, text_list),columns=['heading', 'content'] )

In [ ]:
text_df.head(30)

,heading,content
0,NETWORK 1 FINANCIAL C...,
1,AFFILIATE OFFICE AG...,THIS AGREEMENT is entered into by and b...
2,RECITALS,"WHEREAS, Network 1 Financial, Inc. (""Network 1..."
3,ARTICLE I,
4,[OBLIGATIONS OF AFFILIATES],
5,OBLIGATIONS OF AFFILIATES,1.01 CONTRACTORS. Affiliate shall use i...
6,Section 2.01,
7,[CONTRACTORS],. All such Contractors must process\nMerchan...
8,ARTICLE II,
9,[RIGHTS OF NETWORK 1],


In [ ]:
text_df.tail(30)

,heading,content
17,COMPENSATION,3.01 AMOUNT. As compensation for Affiliat...
18,Section 1.01,
19,[CONTRACTORS],.\nAffiliate shall receive the buy rate with...
20,[RETENTION OF CONTRACTORS],and 2.03 (c). The Affiliate's Fees outli...
21,Section 3.01,
22,[AMOUNT],unless and until Network 1 receives the\nabov...
23,ARTICLE IV,
24,[RESTRICTIONS ON AFFILIATE],
25,RESTRICTIONS ON AFFILIATE,"4.01 RESTRICTIONS. Affiliate shall not, w..."
26,ARTICLE V,


In [ ]:
text_df = text_df.reset_index(drop=True)

for i in range(len(text_df)-1):
    if text_df.at[i, 'content'] == '' and text_df.at[i, 'heading'] != '':
        text_df.at[i+1, 'heading'] = text_df.at[i, 'heading'] + '. ' + text_df.at[i+1, 'heading']
        text_df.drop(i, inplace=True)
  
    elif text_df.at[i, 'heading'] == '' and text_df.at[i, 'content'] != '':
        text_df.at[i+1, 'content'] = text_df.at[i, 'content'] + '. ' + text_df.at[i+1, 'content']
        text_df.drop(i, inplace=True)


In [ ]:
text_df

,heading,content
1,NETWORK 1 FINANCIAL C...,THIS AGREEMENT is entered into by and b...
2,RECITALS,"WHEREAS, Network 1 Financial, Inc. (""Network 1..."
5,ARTICLE I. [OBLIGATIONS OF AFFILIATES]. OBLI...,1.01 CONTRACTORS. Affiliate shall use i...
7,Section 2.01. [CONTRACTORS],. All such Contractors must process\nMerchan...
10,ARTICLE II. [RIGHTS OF NETWORK 1]. RIGHTS O...,2.01 CONTRACTORS. Network 1 shall have...
12,Section 1.01. [CONTRACTORS],. All individuals identified by Affiliate\nsh...
13,Section 1.02,[LOCAL OFFICE (AFFILIATE OFFICE)]\n.\n2.03 ...
14,[RETENTION OF CONTRACTORS],(iv) 18\nmonths have lapsed under this Agr...
17,ARTICLE III. [COMPENSATION]. COMPENSATION,3.01 AMOUNT. As compensation for Affiliat...
19,Section 1.01. [CONTRACTORS],.\nAffiliate shall receive the buy rate with...


In [ ]:
# Define a regex pattern to detect section numbers
section_pattern = re.compile(r'^\s*(\d*\.\s+|\d*\.|M{0,3}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})\.\s+|M{0,3}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})\.)+')

# Define an empty list to store the results
result = []

# Iterate over the rows of the dataframe
for _, row in text_df.iterrows():
  
    # Split the content column into individual sentences
    sentences = row['content'].split('\n')

    # Define a variable to store the heading
    heading = None
    
    # Iterate over the sentences
    for sentence in sentences:
        # Check if the sentence is a section heading
        if section_pattern.match(sentence):
            # If it is, set the heading variable to the first sentence
            heading = sentence
            
            # Append the section heading as a new row to the result list
            result.append({
                'heading': heading,
                'content': ''
            })
        elif heading:
            # If there is a heading, append the sentence to the content of the previous section
            result[-1]['content'] += sentence + '\n'
        
        else:
            # If there is no heading, keep the original heading and add the sentence to the content
            if any(r['heading'] == row['heading'] for r in result):
                # If a row with the same heading already exists, append the content to its existing content
                result[-1]['content'] += '\n' + sentence
            else:
                # Otherwise, create a new row with the original heading and the sentence as the content
                result.append({
                    'heading': row['heading'],
                    'content': sentence
                })

# Convert the result list to a new dataframe
new_df = pd.DataFrame(result)

In [ ]:
new_df.head(50)

,heading,content
0,NETWORK 1 FINANCIAL C...,THIS AGREEMENT is entered into by and b...
1,RECITALS,"WHEREAS, Network 1 Financial, Inc. (""Network 1..."
2,ARTICLE I. [OBLIGATIONS OF AFFILIATES]. OBLI...,to commercial businesses the Services of Netw...
3,Section 2.01. [CONTRACTORS]. All such Con...,Merchant applications and transactions exclusi...
4,Section 2.01. [CONTRACTORS]1.02 LOCAL O...,"maintain such offices (the ""Local Office""), as..."
5,Section 2.01. [CONTRACTORS]1.03 REQUIRE...,must also be registered representative th...
6,Section 2.01. [CONTRACTORS]1.04 PURCHAS...,leased or otherwise provided by Affiliate to M...
7,Section 2.01. [CONTRACTORS]1.05 NETWORK...,submit all Merchant Agreements procured by...
8,Section 2.01. [CONTRACTORS]1.06 RESTRIC...,"only to bona fide and lawful retail, MO/T..."
9,ARTICLE II. [RIGHTS OF NETWORK 1]. RIGHTS O...,"accept, not accept, terminate or otherwise de..."


In [ ]:
new_df.tail(50)

,heading,content
0,NETWORK 1 FINANCIAL C...,THIS AGREEMENT is entered into by and b...
1,RECITALS,"WHEREAS, Network 1 Financial, Inc. (""Network 1..."
2,1.01 CONTRACTORS. Affiliate shall use i...,to commercial businesses the Services of Netw...
3,. All such Contractors must process,Merchant applications and transactions exclusi...
4,1.02 LOCAL OFFICE (AFFILIATE OFFICE). Aff...,"maintain such offices (the ""Local Office""), as..."
5,1.03 REQUIRED REGISTRATION. All of the...,must also be registered representative th...
6,1.04 PURCHASE AND LEASE OF EQUIPMENT. ...,leased or otherwise provided by Affiliate to M...
7,1.05 NETWORK 1 TO APPROVE ALL MERCHANT...,submit all Merchant Agreements procured by...
8,1.06 RESTRICTED MERCHANTS AND MERCHANT A...,"only to bona fide and lawful retail, MO/T..."
9,2.01 CONTRACTORS. Network 1 shall have...,"accept, not accept, terminate or otherwise de..."


In [ ]:
new_df['content'][1]

'WHEREAS, Network 1 Financial, Inc. ("Network 1") provides through various Member\nBank(s)  ("Member"),  VISA  and  MasterCard  processing  and  related  payment\nprocessing  services  ("Services")  to merchant(s) ("Merchant(s)") in accordance\nwith  the  terms  of  certain  Agreement (s) between Network 1, Member and other\nsettlement/transaction  processing  providers?  and\nWHEREAS,  the  Network  1  desires  to  locate individuals to market Services as\nContractor(s)  (the  "Contractors")  of  Network  1?\nWHEREAS,  Affiliate  desires  to  establish  an  "Affiliate Office" on behalf of\nNetwork 1 and Affiliate to market the Services of Network 1 and its subsidiaries\nand  to  locate Contractors on behalf of Network 1, Member, and Affiliate and to\nprovide  a  "Local  Office"  for  such  Contractors  on  the  terms  and for the\nconsideration  set  forth  herein?  And\nWHEREAS,  Affiliate  requires  a  referral  arrangement  while  Affiliate  is\nnegotiating an ISO sponsorship agree

In [ ]:
new_df = new_df.reset_index(drop=True)

for i in range(len(new_df)-1):
    if new_df.at[i, 'content'] == '' and new_df.at[i, 'heading'] != '':
        new_df.at[i+1, 'heading'] = new_df.at[i, 'heading'] + '. ' + new_df.at[i+1, 'heading']
        new_df.drop(i, inplace=True)
  
    elif new_df.at[i, 'heading'] == '' and new_df.at[i, 'content'] != '':
        new_df.at[i+1, 'content'] = new_df.at[i, 'content'] + '. ' + new_df.at[i+1, 'content']
        new_df.drop(i, inplace=True)

In [ ]:
new_df.head()

,heading,content
0,EXHIBIT 10.1. BUSINE...,"This Business Affiliate Agreement (""Agree..."
1,RECITALS,"WHEREAS, the Business Affiliate wishes..."
2,AGREEMENT,1. Definitions. A Confidential Inform...
3,"Source: UNION DENTAL HOLDINGS, INC., 8-K/A, 2/...",4. Mutual Representation and Warranty. ...
4,[Responsibilities of the Parties],and\n 3(h)\n and 6\n shall survive the te...


In [ ]:
new_df['heading'][2]

'                                    AGREEMENT'